<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/%D0%9A%D0%B5%D0%B9%D1%812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gspread  # Для работы с Google Sheets
from oauth2client.service_account import ServiceAccountCredentials  # Для авторизации в Google API
from datetime import datetime  # Для работы с датами

def generate_report(sheet1_name, sheet2_name, sheet3_name): # sheet_name - название листа

    SPREADSHEET_ID = "1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U" # ID таблицы в Google Sheets (из URL)

    # Области доступа для Google API (чтение таблиц и доступ к Google Drive)
    scope = [
        'https://www.googleapis.com/auth/spreadsheets.readonly',
        'https://www.googleapis.com/auth/drive'
    ]
    try:
        print("Авторизация в Google API")
        creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope) # Загрузка учетных данных сервисного аккаунта
        client = gspread.authorize(creds) # Создание клиента для работы с Google Sheets
        spreadsheet = client.open_by_key(SPREADSHEET_ID) # Открытие таблицы по ID

        print("Проверка доступности листов")
        all_sheets = spreadsheet.worksheets()  # Получение списка всех листов в таблице
        sheet_names = [ws.title for ws in all_sheets]
        print(f"Доступные листы в таблице: {sheet_names}")

        requested_sheets = [sheet1_name, sheet2_name, sheet3_name]
        for sheet_name in requested_sheets: # Проверка наличия запрошенных листов
            if sheet_name not in sheet_names:
                print(f"❌ Ошибка: Лист '{sheet_name}' не найден!")
                print(f"Доступные листы: {sheet_names}")
                return

        print("\n Загрузка данных из таблицы...")
        sheet1 = spreadsheet.worksheet(sheet1_name) # Загрузка с листа
        sheet1_data = sheet1.get_all_records()  # Возврат списка словарей
        #sheet1_data = [{'student_id': 1, 'name': 'Иванов И.И.', 'installment': 'Y', 'grade': 'A'}, ....]
        sheet2 = spreadsheet.worksheet(sheet2_name)
        sheet2_data = sheet2.get_all_records()
        sheet3 = spreadsheet.worksheet(sheet3_name)
        sheet3_data = sheet3.get_all_records()

        current_date = datetime(2023, 3, 1) # Дата расчета

        if not sheet1_data: # Проверка листа на пустоту
            print("❌ {sheet1_name} пуст!")
            return

        first_row = sheet1_data[0] # Первая строка
        installment_col = None # Переменная для хранения заголовка
        for key in first_row.keys(): # Перебор всех ключей в первой строке
            if 'installment' in key.lower():  # Ищем колонку installment для определения рассрочки
                installment_col = key
                break

        if not installment_col:
            print("❌ Не найдена колонка с installment!")
            return

        name_col = None # Колонка с ФИО
        for key in first_row.keys():
            if any(word in key.lower() for word in ['name', 'фио', 'студент']): # Поиск колонки с ФИО
                name_col = key
                break

        print("\n Фильтрация студентов с installment='Y'...")
        students_with_y = [] # Отбор студентов с рассрочкой
        for student in sheet1_data: # перебор студентов в списке словарей
            installment_value = student.get(installment_col, '')
            if str(installment_value).strip().upper() == 'Y': # Строка без пробелов в верх. регистре
                student_name = student.get(name_col, '') if name_col else ''
                if student_name and str(student_name).strip(): # Проверка наличия ФИО
                    students_with_y.append(student)
        print(f"Найдено {len(students_with_y)} студентов с рассрочкой")

        sheet2_by_id = {} # Словарь для быстрого поиска данных по student_id
        for record in sheet2_data:
            student_id = record.get('student_id') # Берем id студента
            if student_id:
                sheet2_by_id[student_id] = record  # Данные добавляем в словарь

        sheet3_by_id = {} # Словарь для быстрого поиска данных по student_id
        for record in sheet3_data:
            student_id = record.get('student_id') # Берем id студента
            if student_id:
                sheet3_by_id[student_id] = record  # Данные добавляем в словарь

        print("\n Расчет задолженностей...")
        results = []  # Список для хранения результатов расчета

        for student in students_with_y:
            student_id = student.get('student_id')  # Получаем id студента
            if not student_id:
                continue

            student_data_2 = sheet2_by_id.get(student_id) # Получаем данные по датам платежей
            if not student_data_2:
                continue

            # Получаем данные студента из Лист3 (финансы)
            student_data_3 = sheet3_by_id.get(student_id) # Получаем данные о задолженности
            if not student_data_3:
                continue

            last_payment_date_str = student_data_2.get('last_payment_date') # Берем дату последнего платежа из Лист2
            if not last_payment_date_str or str(last_payment_date_str).strip() == '':
                continue

            one_time_payment = student_data_3.get('one-time_payment')  # Получаем сумму разового платежа
            left_to_pay = student_data_3.get('left_to_pay')  # и остаток долга

            if not one_time_payment or not left_to_pay:
                continue

            try:
                payment_date = None
                date_formats = ['%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y', '%m/%d/%Y']

                for date_format in date_formats: # Парсинг даты
                    try:
                        payment_date = datetime.strptime(str(last_payment_date_str).strip(), date_format)
                        break
                    except ValueError:
                        continue

                if not payment_date:
                    continue

                days_diff = (current_date - payment_date).days # Сколько дней прошло с последнего платежа

                if days_diff < 0:
                    continue

                periods = days_diff // 182

                calculated_debt = periods * one_time_payment # Расчет долга

                final_debt = min(calculated_debt, left_to_pay) # Мин из долга и остатка

                if final_debt > 0:
                    student_name = student.get(name_col, f"Студент ID {student_id}") # Получаем имя
                    results.append({   # Добавляем в словарь результатов
                        'student_id': student_id,
                        'name': student_name,
                        'debt': final_debt,
                    })

            except Exception:
                continue

        filename = 'student_debt_report.txt' # Имя выходного файла
        print(f"\n Сохранение результатов в файл {filename}...")

        with open(filename, 'w', encoding='utf-8') as f:
            for result in results:
                name_clean = result['name'].strip() # Убираем пробелы в начале и конце имени

                if not name_clean:
                    continue

                formatted_name = name_clean

                f.write(f"Студент {formatted_name} - долг {int(result['debt'])} рублей\n")

        print(f"Файл сохранен: {filename}")
        print(f"Результаты: {len(results)} студентов с долгом")

    except Exception as e:
        print(f"❌ Прочие ошибки: {e}")



generate_report('Лист1', 'Лист2', 'Лист3')

Авторизация в Google API
Проверка доступности листов
Доступные листы в таблице: ['Лист1', 'Лист2', 'Лист3']

 Загрузка данных из таблицы...

 Фильтрация студентов с installment='Y'...
Найдено 253 студентов с рассрочкой

 Расчет задолженностей...

 Сохранение результатов в файл student_debt_report.txt...
Файл сохранен: student_debt_report.txt
Результаты: 166 студентов с долгом
